In [1]:
import pandas as pd
import sys
import pandarallel
if sys.platform == 'darwin':
    pandarallel.core.CONTEXT = pandarallel.core.multiprocessing.get_context('spawn')
# see https://github.com/nalepae/pandarallel/issues/225
pandarallel.pandarallel.initialize(progress_bar=True)

import agenas_pne_scraper

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
print (f'agenas-pne-scraper version: {agenas_pne_scraper.__version__}')

agenas-pne-scraper version: 0.1.0


In [ ]:
YEAR = 2021

In [ ]:
hospitals_df = agenas_pne_scraper.get_hospital_id_hospital_name_hospitals_df()

hospital_code_legend = hospitals_df.set_index('hospital_id').hospital_name

### Volume indicators
Table
- `strutture/stru_frequenza.php` + `cod_struttura` as querystring

Graph with historical data
- `strutture/grafico1Str5_HC.php` + `cod_struttura` and `ind` as querystring

### Outcome indicators
Table
- `strutture/stru_indicatori.php` + `cod_struttura` and `conf["reg", "prec"]` as querystring. Conf stands for _confronto_, which means compare in italian; "reg" stands for regional mean, "prec" for precedent year.


Graphs

### Waiting time indicators
Table
-`strutture/stru_tempi.php` + `cod_struttura` as querystring

Graphs


### Volume indicators

In [ ]:
volume_indicators = pd.concat(
    hospital_code_legend.index.to_series().parallel_map(agenas_pne_scraper.PNEVolumeIndicatorsDownloader.generate_pandas_mapper(year=YEAR)).to_list(),
    axis=0,
    ignore_index=True
    )
volume_indicators

### Outcome indicators

In [ ]:
outcome_indicators = pd.concat(
    hospital_code_legend.index.to_series().parallel_map(agenas_pne_scraper.PNEOutcomeIndicatorsDownloader.generate_pandas_mapper(year=YEAR, compare='both')).to_list(),
    axis=0,
    ignore_index=True
    )
outcome_indicators

### Waiting Time indicators

In [ ]:
wt_indicators = pd.concat(
    hospital_code_legend.index.to_series().parallel_map(agenas_pne_scraper.PNEWaitingTimeIndicatorsDownloader.generate_pandas_mapper(year=YEAR)).to_list(),
    axis=0,
    ignore_index=True
    )
wt_indicators

### Graphs

In [ ]:
outcome_graphs_indicators = pd.concat(
    outcome_indicators[['hospital_code', 'indicator_id']].drop_duplicates().dropna(how='any').parallel_apply(agenas_pne_scraper.PNEOutcomeGraphsDownloader.generate_pandas_mapper(), axis=1).to_list(),
    axis=0,
    ignore_index=True
    )
outcome_graphs_indicators

In [ ]:
volume_graphs_indicators = pd.concat(
    volume_indicators[['hospital_code', 'indicator_id']].drop_duplicates().dropna(how='any').parallel_apply(agenas_pne_scraper.PNEVolumeGraphsDownloader.generate_pandas_mapper(), axis=1).to_list(),
    axis=0,
    ignore_index=True
    )
volume_graphs_indicators

In [ ]:
wt_graphs_indicators = pd.concat(
    wt_indicators[['hospital_code', 'indicator_id']].drop_duplicates().dropna(how='any').parallel_apply(agenas_pne_scraper.PNEWaitingTimeGraphsDownloader.generate_pandas_mapper(), axis=1).to_list(),
    axis=0,
    ignore_index=True
    )
wt_graphs_indicators

### Merge indicator dfs

In [ ]:
# Rename columns for merging
outcome_indicators = outcome_indicators.rename(columns={'cases':'value'})
wt_indicators = wt_indicators.rename(columns={'cases':'value'})
df = pd.concat([volume_indicators, outcome_indicators, wt_indicators], axis=0)
df.to_csv(f'./PNE_{YEAR}.csv', index=False)

In [ ]:
df = pd.concat([volume_graphs_indicators, outcome_graphs_indicators, wt_graphs_indicators], axis=0)
df.to_csv('./PNE_historical.csv', index=False)